## OpenAI Agents Framework
This notebook demonstrates the use of the OpenAI Agents framework to create a set of subject matter teachers. The agent also incldues a guardrail set to make sure only homework questions are asked.

Let's start by supressing unwanted output.

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

We'll need to load the openai and open ai agents libraries

In [ ]:
%pip install openai-agents

We'll now import the Agent module into our code. We'll also import the guardrail and agent runner which we'll be using and the base model class.  We'll also make sure we have our OpenAI API key set up ready to go.

In [ ]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import os, asyncio
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

We'll define a homework base model and give it two roles plus a guardrail.

In [ ]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)


We'll define the guardrail function

In [ ]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

We'll now define the hand-offs between the agents

In [ ]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

We can now define our asynchronous mainline function

In [ ]:
async def main(question):
   try:
      result = await Runner.run(triage_agent, question)
      print(result.final_output)
   except:
      print("Guardrail triggered - this is not a homework question")

Let's now ask our agent a question.

In [ ]:
if __name__ == "__main__":
    question = "Tell me about how to calulate the diameter of a circle"
    await main(question)

Let's do that again with a question that is not about homework.

In [ ]:
if __name__ == "__main__":
    question = "Tell me about Taylor Swift's upcoming concert schedule"
    await main(question)